<a href="https://colab.research.google.com/github/KanadeSisido/Learning-RecommenderSystems-with-X/blob/main/Matrix_Factorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Matrix Factorization

Netflix Prizeで最も成果を上げた推薦アルゴリズム．
その大枠の解説と再現を行う．

## Matrix Factorization は何をしているのか

MF(Matrix Factorization)では，以下のような行列の分解を考える．
$$
R = PQ^T
$$
$R$は，$(i,j)$成分にユーザ$p_i$がアイテム$q_j$に対して行った評価が書き込まれる．

$P,Q$はそれぞれ$P = [\hat p_0,\hat p_1, \cdots, \hat p_n]$，$Q = [\hat q_0, \hat q_1, \cdots, \hat q_n]$といった具合に，ユーザ$p_i$がもつ潜在ベクトル$\hat p_i$，アイテム$q_i$がもつ潜在ベクトル$\hat q_i$を考え，それを並べた行列$P,Q$に分割することを考える．

ここで，$p_i$はユーザ$p_i$と潜在特徴との関係，$q_i$はアイテム$q_i$と潜在特徴との関係を示している．すなわち，潜在特徴を基底に取ったベクトル空間の元である．

ここで，ベクトルの内積$p_i \cdot q_i$をとることは，同一のベクトル空間上でベクトル同士の親和性の高さを得ることに相当する．

## どうやって$P,Q$を得るのか

$P,Q$の算出は以下の最小化問題に帰着できる．
$$
\min_{P,Q} \sum_{(i,j) \in R}(R_{i,j} - p_i \cdot q_i)^2 + \lambda(\| p_i \|_{L2}^2 + \| q_i \|_{L2}^2)
$$

基本戦略としては，$P,Q$をランダムにとり，積を取った結果から，$L$との誤差を算出し，更新式に入力することで$P,Q$を学習する．

$$
\delta_{i,j} = R_{i,j} - p_i \cdot q_j
$$
更新後の値は以下の通り
$$
p_i= p_i + 2 \alpha \delta q_i \\
q_i= q_i + 2 \alpha \delta p_i
$$


## 実装

$$
P=
\left(
\begin{array}{rrr}
p_1 & p_2 & \cdots & p_n \\
\end{array}
\right)
$$

$$
Q=
\left(
\begin{array}{rrr}
q_1 & q_2 & \cdots & q_n \\
\end{array}
\right)
$$

$$
\hat R = PQ^T
$$

更新式ちょっと変えてみた（コード参照）

In [51]:
import numpy as np

def delta(r: float, p, q):
  return r - np.dot(p, q)

def totalError(R, P, Q, lam):
  [n, m] = R.shape

  err = 0

  for i in range(n):
    for j in range(m):
      if R[i][j] == 0:
        continue
      err += delta(R[i][j], P[i], Q[j]) ** 2
  
  return err

def MF(R, k, maxStep, alpha, lam):

  [n, m] = R.shape

  P = np.random.rand(n, k)
  Q = np.random.rand(m, k)

  print(np.dot(P, Q.T))
  
  for count in range(maxStep):
    for i in range(n):
      for j in range(m):

        if R[i][j] == 0:
          continue

        d = delta(R[i][j], P[i], Q[j])
        Pn = P[i] + alpha * 2 * d * (Q[j] - lam * P[i])
        Qn = Q[j] + alpha * 2 * d * (P[i] - lam * Q[j])

        P[i] = Pn
        Q[j] = Qn

    err = totalError(R, P, Q, lam)

    if count % 10 == 0:
      print("count : ", count , "\tErr : ", err)
  
  return P, Q

a = np.array(
  [
    [5,1,5,3,2],
    [3,0,2,2,4],
    [3,5,0,1,5],
    [2,0,2,3,1],
  ]
)

P, Q = MF(a, 5, 100, 0.1, 0.02)

print(np.dot(P, Q.T))


[[1.60245888 1.64363267 1.63031852 1.77088604 1.56751282]
 [1.22591048 0.89249827 0.9958355  2.20533856 1.12903698]
 [1.08794134 0.83402896 0.85360345 1.27758317 0.57199407]
 [1.47071831 1.05086241 1.14751669 2.16374893 1.02540996]]
count :  0 	Err :  33.31153766438655
count :  10 	Err :  0.4381241682616663
count :  20 	Err :  0.009941946929742365
count :  30 	Err :  0.0003404365912820631
count :  40 	Err :  1.943474036305886e-05
count :  50 	Err :  1.6820862427343429e-06
count :  60 	Err :  1.5420611710208936e-07
count :  70 	Err :  1.3233327491986992e-08
count :  80 	Err :  1.1038138735402451e-09
count :  90 	Err :  9.307446804697184e-11
[[5.00000108 0.99999948 5.00000008 2.99999941 2.00000096]
 [3.00000036 2.5690675  1.99999969 1.99999913 3.99999957]
 [3.00000215 4.9999992  2.53325884 0.9999998  5.00000072]
 [2.         0.66083326 2.00000012 2.99999965 0.99999941]]


hello
